In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import requests

## Importing CSVs

In [2]:
path_death = 'causes_of_death.csv'

df_death = pd.read_csv(path_death)
df_death.head()

,Year,ZIP Code,Causes of Death,Count,Location
0,1999,90002,SUI,1,"(33.94969, -118.246213)"
1,1999,90005,HOM,1,"(34.058508, -118.301197)"
2,1999,90006,ALZ,1,"(34.049323, -118.291687)"
3,1999,90007,ALZ,1,"(34.029442, -118.287095)"
4,1999,90009,DIA,1,"(33.9452, -118.3832)"


In [11]:
df_path = 'irs_income_by_zip_code.csv'

df_income = pd.read_csv(df_path)
df_death.head()

,Year,ZIP Code,Causes of Death,Count,Location
0,1999,90002,SUI,1,"(33.94969, -118.246213)"
1,1999,90005,HOM,1,"(34.058508, -118.301197)"
2,1999,90006,ALZ,1,"(34.049323, -118.291687)"
3,1999,90007,ALZ,1,"(34.029442, -118.287095)"
4,1999,90009,DIA,1,"(33.9452, -118.3832)"


In [3]:
path_population_2000 = 'population_by_zip_2000.csv'

population_2000 = pd.read_csv(path_population_2000)
population_2000.head()

,minimum_age,maximum_age,gender,population,zipcode,geo_id
0,10.0,14.0,female,75,39769,8600000US39769
1,85.0,NaN,female,130,44047,8600000US44047
2,30.0,34.0,female,11,42436,8600000US42436
3,22.0,24.0,male,22,99685,8600000US99685
4,67.0,69.0,male,69,04239,8600000US04239


In [4]:
path_population_2010 = 'population_by_zip_2010.csv'

population_2010 = pd.read_csv(path_population_2010)
population_2010.head()

,population,minimum_age,maximum_age,gender,zipcode,geo_id
0,50,30.0,34.0,female,61747,8600000US61747
1,5,85.0,NaN,male,64120,8600000US64120
2,1389,30.0,34.0,male,95117,8600000US95117
3,231,60.0,61.0,female,74074,8600000US74074
4,56,0.0,4.0,female,58042,8600000US58042


## First Data-Set Cleaning

In [5]:
df_death_final = df_death[['ZIP Code','Causes of Death','Count']]

In [6]:
idx = df_death.groupby(['ZIP Code'])['Count'].transform(max) == df_death['Count']

In [7]:
di = df_death[idx]

df_u = di.drop_duplicates(subset=['ZIP Code', 'Count'], keep='first', inplace=False)
df_final = df_u.loc[df_u['Count'] > 20]
df_final_cause_death = df_final[['ZIP Code', 'Causes of Death', 'Count']]
df_final_cause_death.tail()

,ZIP Code,Causes of Death,Count
231942,95608,HTD,249
231962,92543,HTD,292
231990,92653,HTD,380
231991,99998,HTD,388
232017,99999,OTH,1003


 some finding from the first data set

In [8]:
zip_num_death = df_death['ZIP Code'].nunique()
print(f"There are {zip_num_death} unique Zipcodes in the 'Cause of Death' csv")

There are 1785 unique Zipcodes in the 'Cause of Death' csv


In [9]:
diseases_total = df_death['Causes of Death'].nunique()
print(f"There are {diseases_total} unique diseases in the data-set")

There are 14 unique diseases in the data-set


## Second Data Set Cleaning

In [12]:
#added number of returns to calculate unemployment **we can get another data set that shows 
#the population per zip code 
#if we can find one
df_final_income = df_income[['ZIPCODE', 'Avg AGI', 'Number of returns']]
df_final_income.head()

,ZIPCODE,Avg AGI,Number of returns
0,0,51.963410,2022380
1,35004,51.832454,4930
2,35005,38.905152,3300
3,35006,47.400000,1230
4,35007,53.687073,11990


Bining

In [13]:
bins = [0, 50, 70, 100]
labels = ['low', 'middle', 'high']


In [14]:
df_final_income["income_class"] = pd.cut(df_final_income["Avg AGI"], bins, labels=labels)
df_final_income.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ZIPCODE,Avg AGI,Number of returns,income_class
0,0,51.963410,2022380,middle
1,35004,51.832454,4930,middle
2,35005,38.905152,3300,low
3,35006,47.400000,1230,low
4,35007,53.687073,11990,middle


In [15]:
zip_codes_with_zeros = df_final_income.loc[df_final_income["ZIPCODE"] == 0] #no n

In [16]:
df_final_income_dropped_zeroes = df_final_income[df_final_income.ZIPCODE != 0]
df_final_income_dropped_zeroes.head(4)

,ZIPCODE,Avg AGI,Number of returns,income_class
1,35004,51.832454,4930,middle
2,35005,38.905152,3300,low
3,35006,47.400000,1230,low
4,35007,53.687073,11990,middle


In [17]:
zip_num_income = df_final_income['ZIPCODE'].nunique()
print(f"There are {zip_num_income} unique zipcodes in the 'income' csv")

There are 27690 unique zipcodes in the 'income' csv


## Third Data Set Cleaning

In [ ]:
population_2000_clean = population_2000[['population', 'zipcode']]

In [ ]:
population_2000_clean.head()

## Fourth Data Set Cleaning

In [ ]:
population_2010_clean = population_2010[['population', 'zipcode']]

In [ ]:
population_2010_clean.head()

## Web Scraping

In [ ]:
#states based on the first number of the zipcode

In [ ]:
url = "https://smartystreets.com/docs/zip-codes-101"

html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

In [ ]:
objectt = soup.find_all('table')
objectt

In [ ]:
states = []
numbers = []

for i in objectt:
    for x in i:
        for z in x:
            for m in z:
                for c in m:
                    print("-----------")
                    print(c)

In [ ]:
list1 = []
for i in objectt:
    for x in i:
        for z in x:
            for m in z:
                for c in m:
                    #print("-----------")
                    list1.append(c)
list1 

In [ ]:
#figure this out

In [ ]:
list_of_words = []

for sentence in draft:
    word = ''
    for letter in sentence:
        if letter != ',' and letter != ' ':
            word += letter
        else:
            break
    print(word)        
    list_of_words.append(word)
    word = ''

# ORM Set Up

In [ ]:
#conn = "<insert user name>:<insert password>@localhost:5432/customer_db" 

In [ ]:
conn = "postgres:postgres@localhost:5432/zip_codes"
engine = create_engine(f'postgresql://{conn}')

In [ ]:
engine.table_names() #checking for tables, we can also do that by checking the PG Adming

## Loading to DB

 load first Data Set

In [ ]:
df_final_cause_death.to_sql(name='cause_of_death', con=engine, if_exists='append', index=False)

 load second Data Set

In [ ]:
df_final_income_dropped_zeroes.to_sql(name='income_by_zip', con=engine, if_exists='append', index=False)

 load third Data Set

In [ ]:
population_2000_clean.to_sql(name='population_by_zip_code_2000', con=engine, if_exists='append', index=False)

 load fourth Data Set

In [ ]:
population_2010_clean.to_sql(name='population_by_zip_code_2010', con=engine, if_exists='append', index=False)

In [ ]:
engine.table_names()